In [1]:
import math
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
df_time = pd.read_csv('Datasets/Vessels_ETA_ATA.csv')# Write your code for importing the csv file here

In [3]:


df_time['ETA']= pd.to_datetime(df_time['ETA'],format="%Y-%m-%d %H:%M")
df_time['ETD']= pd.to_datetime(df_time['ETD'],format="%Y-%m-%d %H:%M")
df_time['ATA']= pd.to_datetime(df_time['ATA'],format="%Y-%m-%d %H:%M")
df_time['Recording']= pd.to_datetime(df_time['Recording'], format="%Y-%m-%d %H:%M")

df_detail = pd.read_csv('Datasets/Clarksons_World_Containership_Fleet_2021_Details.csv')
df_detail = df_detail.rename(columns={'Name': 'Vessel_Name'})
df_time = df_time.rename(columns={'Vessel Name': 'Vessel_Name'})

df_time['Vessel_Name'] = df_time['Vessel_Name'].str.lower()
df_detail['Vessel_Name'] = df_detail['Vessel_Name'].str.lower()





In [4]:
df_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907 entries, 0 to 906
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Ship_ID            907 non-null    object        
 1   Vessel_Name        907 non-null    object        
 2   Service            907 non-null    object        
 3   Phase              907 non-null    object        
 4   ETA                907 non-null    datetime64[ns]
 5   ATA                907 non-null    datetime64[ns]
 6   Berth              907 non-null    object        
 7   Side to            907 non-null    object        
 8   ETD                907 non-null    datetime64[ns]
 9   Next port of call  907 non-null    object        
 10  Ref                907 non-null    object        
 11  Recording          907 non-null    datetime64[ns]
 12  SN_No.             907 non-null    int64         
 13  Delay              907 non-null    float64       
dtypes: datetim

In [5]:

pd.set_option('display.max_columns', None)

new_df= pd.merge(df_time, df_detail, how='left', on='Vessel_Name')


In [6]:
new_df.head()

,Ship_ID,Vessel_Name,Service,Phase,ETA,ATA,Berth,Side to,ETD,Next port of call,Ref,Recording,SN_No.,Delay,Type,Size,Unit,Dwt,GT,Flag,Built,Month,Builder,Owner Group,Status,Alternative Fuel Types,SOx Scrubber Status,Eco - Electronic Engine
0,AL DAHNA - C07/D07,al dahna,FE2,ARRIVED,2021-03-04 16:00:00,2021-03-09 02:58:00,SCT5,Starboard,2021-03-07 04:00:00,"Hamburg, Germany",C07/D07,2021-02-20,2577,6418.0,Container,"18,800",TEU,"199,744","195,636",Malta,2016.0,1.0,Hyundai HI (Ulsan),Hapag-Lloyd,Idle,NaN,NaN,Eco – Electronic Engine Modern
1,AL DAHNA - C23/D23,al dahna,FE2,ARRIVED,2021-07-21 07:30:00,2021-07-21 23:36:00,SCT5,Starboard,2021-07-24 07:30:00,"Hamburg, Germany",C23/D23,2021-07-10,1261,966.0,Container,"18,800",TEU,"199,744","195,636",Malta,2016.0,1.0,Hyundai HI (Ulsan),Hapag-Lloyd,Idle,NaN,NaN,Eco – Electronic Engine Modern
2,AL DAHNA - C26/D26,al dahna,FE2,ARRIVED,2020-08-06 19:00:00,2020-08-07 20:40:00,SCT5,Starboard,2020-08-08 19:00:00,"Hamburg, Germany",C26/D26,2020-07-25,3686,1540.0,Container,"18,800",TEU,"199,744","195,636",Malta,2016.0,1.0,Hyundai HI (Ulsan),Hapag-Lloyd,Idle,NaN,NaN,Eco – Electronic Engine Modern
3,AL DAHNA - C40/D40,al dahna,FE2,ARRIVED,2020-11-18 10:00:00,2020-11-22 07:45:00,SCT5,Starboard,2020-11-20 10:00:00,"Hamburg, Germany",C40/D40,2020-11-05,785,5625.0,Container,"18,800",TEU,"199,744","195,636",Malta,2016.0,1.0,Hyundai HI (Ulsan),Hapag-Lloyd,Idle,NaN,NaN,Eco – Electronic Engine Modern
4,AL HILAL - O27/O27,al hilal,AL2,ARRIVED,2021-07-05 07:30:00,2021-07-10 15:48:00,SCT3,Starboard,2021-07-05 19:30:00,"Rotterdam, Netherlands",O27/O27,2021-06-20,2807,7698.0,Container,"6,921",TEU,"85,437","75,579",Liberia,2008.0,9.0,Hyundai HI (Ulsan),Hapag-Lloyd,In Service,NaN,NaN,Eco – Electronic Engine


In [7]:


new_df['Alternative Fuel Types'] = new_df['Alternative Fuel Types'].fillna('0')
new_df['SOx Scrubber Status'] = new_df['SOx Scrubber Status'].fillna('0')
new_df['Eco - Electronic Engine'] = new_df['Eco - Electronic Engine'].fillna('0')

new_df = new_df.rename(columns={'Alternative Fuel Types': 'Alternative_Fuel','SOx Scrubber Status': 'SOx_Scrubber', 'Eco - Electronic Engine': 'Eco_Engine' })



In [8]:
new_df['Days_Before_ETA_Info'] =  new_df['ETA'] - new_df['Recording']


new_df['Days_Before_ETA_Info']=new_df['Days_Before_ETA_Info'].astype('timedelta64[m]') #Not using[D] cuz its rounds down
new_df['Days_Before_ETA_Info']=new_df['Days_Before_ETA_Info']/1440 #60*24

new_df['Days_Before_ETA_Info']=new_df['Days_Before_ETA_Info'].round(0)

In [9]:
new_df.drop(['Phase', 'ETD','Recording','SN_No.','Type','Unit','Status'], axis=1,inplace=True)

In [10]:
new_df.dropna(inplace=True) #remove rows with non value and save,

In [11]:
new_df.to_csv(r'Datasets/Vessel_Detail_Merged.csv', index = False, header = True)

In [ ]:
new_df.info()

In [ ]:
#new_df['Size'] = new_df['Size'].str.replace(',', '').astype('int32')
#new_df['Dwt'] = new_df['Dwt'].str.replace(',', '').astype('int32')
#new_df['GT'] = new_df['GT'].str.replace(',', '').astype('int32')



#new_df["Size"]=new_df["Size"].astype('int32')

In [ ]:
new_df.corr()